In [1]:
import tensorflow as tf
from tqdm import tqdm
import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding,BatchNormalization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
import inception_v3
from tqdm import tqdm
import math

from NIC import model,greedy_inference_model,image_dense_lstm,text_emb_lstm

Using TensorFlow backend.


In [2]:
fp2="data/senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)



In [3]:
fp3= "data/captions_train2014.json"
with open(fp3,'r') as f3:
    mscoco_data= json.load(f3)

In [4]:
all_img_name_vector=[]
all_captions=[]
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH_TRAIN=PATH_DIR + '/data/train2014/'
for annot in mscoco_data['annotations']:
    caption =  annot['caption']
   
    caption = caption.rstrip()
    caption = caption.lower()
    caption = [x.replace(".", "") for x in caption]
    caption = "".join(caption)
    caption = caption.rstrip()
    caption = "".join(caption)
    
    image_id = annot['image_id']
    full_coco_image_path = PATH_TRAIN + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)
#     
# train_captions, img_name_vector = shuffle(all_captions,
#                                           all_img_name_vector,
#                                           random_state=1)
num_examples = 400000
train_captions = all_captions[:num_examples]
img_name_vector = all_img_name_vector[:num_examples]

In [5]:
fp4= "sentence_dict2.json"
with open(fp4,"r") as f4 :
    sentiment_dict= json.load(f4)
    
    

In [6]:
print(train_captions[0]+"#")

a very clean and well decorated empty bathroom#


In [7]:
# img_vec
sentiment_vec=[]
caption_vec=[]
error=0
for i in range(num_examples):
    cap= train_captions[i]
    if cap in sentiment_dict.keys():
        senti= sentiment_dict[cap]
        
    else:
        senti=0
        error+=1
        # print(cap)
    cap= '<start> ' + cap +' <end>'
    caption_vec.append(cap)
    sentiment_vec.append(senti)
    
print(error)

25687


In [8]:
# train_num=30000
image_name_train= img_name_vector[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]
print(caption_train[0])


<start> a very clean and well decorated empty bathroom <end>


In [9]:

        
# senticap_list=[]
# senticap_list_raw=[]
# l=0
# for img_obj in senticap_data['images']:
#     # if (l>20):
#     #     break
#     # l+=1
#     img_name= img_obj['filename']
#     for sent in img_obj['sentences']:
#         sentiment= sent['sentiment']
#         raw= sent['raw'].lower()
#         w0= raw.rstrip()
#         w0 = [x.replace(".", "") for x in w0]
#         w0= [x.replace(",","") for x in w0]
#         w0= [x.replace("'","") for x in w0]
#         w0= [x.replace("?","") for x in w0]
#         w0="".join(w0)
#         senticap_list.append((img_name,sentiment,w0))
#         senticap_list_raw.append((img_name,w0))
        
        

In [10]:
"""write to cap file for nlp treebank"""
# filename='new_cap.txt'
# with open(filename,'w') as f4:
#     for item in senticap_list_raw:
#         w0= item[1]
#         w0= w0.lower()
# 
#         f4.write(w0+" .\n")
        

'write to cap file for nlp treebank'

In [4]:
"""read from output of nlp treebank"""

# fname="new_out.txt"
# i=0
# l1=10000
# senti_score=[]
# senti_avg=[]
# error=0
# senti_var=[]
# train_caption=[]
# senti_cap_dict={}
# with open(fname) as f1:
#     while(i<l1):
#         w1= f1.readline().strip()
#         if not w1:
#             break
#         w2= f1.readline().strip()
#         w1 = [x.replace(".", "") for x in w1]
#         w1= [x.replace(",","") for x in w1]
#         w1= [x.replace("'","") for x in w1]
#         w1= [x.replace("?","") for x in w1]
#         w1= "".join(x for x in w1)
#         w1= w1.rstrip()
#         w1= "".join(x for x in w1)
# 
#         if w2.islower():
#             w1=w2
#             w2=f1.readline().strip()
#         a= 0
#         if w2=="Negative":
#             a= -1
#         elif w2=="Positive":
#             a= 1
#         elif w2=="Very negative":
#             a= -1
#         elif w2=="Very positive":
#             a= 1
#         elif w2=="Neutral":
#             a= 0
#         else:
#             a=0
#             error+=1
#             print(w2)
#         senti_score.append(a)
#         train_caption.append(w1)
#         senti_cap_dict[w1]=a
#         i+=1
# # print(senti_score[0:500] )
# print(len(train_caption),"train-cap_length")
# print(len(senti_score),"train-senti_score")

" .
8861 train-cap_length
8861 train-senti_score


In [5]:
# PATH_DIR= '/home/yiyang/project/caption/caption'
# PATH= PATH_DIR+'/data/val2014/'

In [6]:
# image_vec=[]
# sentiment_vec=[]
# caption_vec=[]
# 
# for item in senticap_list:
#     img= item[0]
#     if item[1]==1:
#         senti=1
#     elif item[1]==0:
#         senti=-1
#     else:
#         senti=0
#         
#     caption= item[2]
#     # print(caption+"#")
#     if caption in senti_cap_dict.keys():
#         new_senti= senti_cap_dict[caption]
#     else:
#         new_senti= 0
#     img= PATH + img 
#     caption= '<start> ' + caption + ' <end>'
#     image_vec.append(img)
#     sentiment_vec.append(new_senti)
#     caption_vec.append(caption)
    

In [7]:
# num_examples = 8861
# start=0
# 
# image_name_train= image_vec[:num_examples]
# senti_train= sentiment_vec[:num_examples]
# caption_train= caption_vec[:num_examples]
# print(caption_train[8000])

<start> a plane sitting on the runway in a clear field during a beautiful sunset <end>


In [11]:
print(img_name_vector[30000])
# img=load_image(image_name_train[0])
img=cv2.imread(img_name_vector[30000]) 
print(train_captions[30000])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


/home/yiyang/project/caption/caption/data/train2014/COCO_train2014_000000440087.jpg
person riding a bicycle with a mask on and a city street


In [12]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    return img


In [13]:
units= 512
embedding_size=511
BATCH_SIZE=80
vocab_size=12000
max_length=20

In [14]:
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_train)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)

print(train_seqs[0])
print(cap_vector_train[0])



[3, 2, 145, 494, 10, 619, 421, 276, 57, 4]
[  3   2 145 494  10 619 421 276  57   4   0   0   0   0   0   0   0   0
   0   0   0]


In [15]:
print(tokenizer.index_word[9999])

wouldn't


In [11]:

# base_model=inception_v3.InceptionV3(include_top=True,weights=None)
# weights_path = 'data/image_net.h5'
# # image_model = tf.keras.applications.InceptionV3(include_top=False,
# #                                                 weights='imagenet')
# base_model.load_weights(weights_path)
# print(base_model.output_shape)
# for layer in base_model.layers[:312]:
#     layer.trainable = False
    

In [15]:

# new_input = base_model.layers[0].input
# hidden_layer = base_model.get_layer('avg_pool').output
# 
# image_model = keras.Model(new_input, hidden_layer)
# print(image_model.output_shape)

(None, 2048)


In [16]:
# start=0
train_num=20000

In [17]:
def batch_generator(batch_size,units,vocab_size,train_num,start):
    input_imgs=[]
    # start=0
    # print(start,'start')
    for j in tqdm(range(train_num)):
        img= load_image( image_name_train[start+j])
        input_imgs.append(img)
    input_imgs= np.array(input_imgs)
    input_sentis= np.array(senti_train[start:start+train_num])
    input_sequences= np.array(cap_vector_train[start:start+train_num])
    
    # print(input_sequences.shape,"input_seq")
    target= input_sequences[:,1:]
    
    # target= np.expand_dims(target,axis=-1)
    # print(target.shape,"target")
    state_c= np.zeros(shape=(train_num,512))
    state_h= np.zeros(shape=(train_num,512))
    
    while True:
        for i in range(0,train_num,batch_size):
            img_features=input_imgs[i:i+batch_size,:]
            seqs= input_sequences[i:i+batch_size,0:max_length]
            state_c= np.zeros([batch_size,units])
            state_h= np.zeros([batch_size,units])
            sentiment= input_sentis[i:i+batch_size]
            
            
            target_seq= input_sequences[i:i+batch_size,1:]
            y= keras.utils.to_categorical(target_seq,vocab_size)
            # print(y.shape,"y-shape")
            yield ([img_features,seqs,sentiment,state_c, state_h],y)



In [18]:
start=50000
train_num=20000
generator_train= batch_generator(BATCH_SIZE,units,vocab_size,train_num,start)

In [19]:
start2=90000
train_num2=2000
generator_dev= batch_generator(BATCH_SIZE,units,vocab_size,train_num2,start2)

In [20]:
reg=1e-4
decay=0
lr=0.001
initial_epoch=0

NIC_model=model(vocab_size,max_length,reg)
# complete_model= keras.Model(image_model.input,NIC_model.output)
NIC_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])


W1006 10:50:31.743468 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1006 10:50:31.754906 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1006 10:50:31.756759 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1006 10:50:31.766759 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W1006 10:50:31.767105 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [17]:

NIC_model.fit_generator(generator_train, steps_per_epoch=30000//BATCH_SIZE,
                        epochs=2,
                        verbose=2, 
                        initial_epoch = initial_epoch)

NIC_model.save_weights("models/NIC_model5.h5")

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 24/40000 [00:00<02:47, 238.61it/s]

Epoch 1/2
50000 start


  0%|          | 50/40000 [00:00<02:44, 243.00it/s]

  0%|          | 77/40000 [00:00<02:41, 247.71it/s]

...

100%|██████████| 40000/40000 [02:47<00:00, 237.07it/s]

100%|██████████| 40000/40000 [02:47<00:00, 239.00it/s]

 - 250s - loss: 5.1405 - acc: 0.2287


Epoch 2/2


 - 76s - loss: 3.9068 - acc: 0.3527


In [32]:
start=320000
generator_train= batch_generator(BATCH_SIZE,units,vocab_size)

In [19]:
NIC_model.load_weights("models/NIC_model5.h5")
NIC_model.fit_generator(generator_train, steps_per_epoch=30000//BATCH_SIZE,
                        epochs=10,
                        verbose=2, 
                        validation_data=generator_dev,
                        validation_steps=100,
                        initial_epoch = 0)

NIC_model.save_weights("models/NIC_model5.h5")


Epoch 1/10


 - 94s - loss: 2.8062 - acc: 0.4935 - val_loss: 3.3849 - val_acc: 0.4274


Epoch 2/10


 - 94s - loss: 2.7884 - acc: 0.4964 - val_loss: 3.3849 - val_acc: 0.4290


Epoch 3/10


...

 - 95s - loss: 2.6683 - acc: 0.5149 - val_loss: 3.4527 - val_acc: 0.4271


Epoch 9/10


 - 95s - loss: 2.7015 - acc: 0.5137 - val_loss: 3.3559 - val_acc: 0.4338


Epoch 10/10


 - 95s - loss: 2.6894 - acc: 0.5142 - val_loss: 3.3485 - val_acc: 0.4404


In [20]:
NIC_inference = greedy_inference_model(vocab_size, max_length)
NIC_inference.load_weights("models/NIC_model5.h5", by_name = True, skip_mismatch=True)

In [21]:
test_imgs=[]
num_test=1000
start=10000
for j in tqdm(range(start,start+num_test)):
    img= load_image( image_name_train[j])
    test_imgs.append(img)
test_imgs= np.array(test_imgs)
test_sentis= np.array(senti_train[start:start+num_test])
test_sequences= np.array(cap_vector_train[start:start+num_test])
# test_seq= np.ones(shape=(500,1))
# test_seq= 3* test_seq
# test_out=[]


  0%|          | 0/1000 [00:00<?, ?it/s]

  2%|▏         | 22/1000 [00:00<00:04, 215.00it/s]

  4%|▍         | 45/1000 [00:00<00:04, 218.87it/s]

  7%|▋         | 69/1000 [00:00<00:04, 223.37it/s]

  9%|▊         | 86/1000 [00:00<00:04, 204.05it/s]

...

 92%|█████████▏| 915/1000 [00:03<00:00, 245.77it/s]

 94%|█████████▍| 941/1000 [00:03<00:00, 246.28it/s]

 97%|█████████▋| 968/1000 [00:04<00:00, 251.87it/s]

 99%|█████████▉| 994/1000 [00:04<00:00, 248.94it/s]

100%|██████████| 1000/1000 [00:04<00:00, 239.68it/s]

In [22]:
N = test_imgs.shape[0]

startseq = np.repeat([tokenizer.word_index['<start>']], N)
a0 = np.zeros([N, units])
c0 = np.zeros([N, units])

# output dims: [32, N, 7378]
y_preds = np.array(NIC_inference.predict([test_imgs, startseq,test_sentis, a0, c0], verbose = 1))

# output dims: [N, 32, 7378]
y_preds = np.transpose(y_preds, axes = [1,0,2])

sequences = np.argmax(y_preds, axis = -1)
sents = tokenizer.sequences_to_texts(sequences)

# if post_process:
    # post processing: 'endseq'
sents_pp = []
for sent in sents:
    if '<end>' in sent.split():
        words = sent.split()
        sents_pp.append(' '.join(words[:words.index('<end>')]))
    else:
        sents_pp.append(sent)
sents = sents_pp
    

  32/1000 [..............................] - ETA: 1:48

  64/1000 [>.............................] - ETA: 54s 

  96/1000 [=>............................] - ETA: 35s

 128/1000 [==>...........................] - ETA: 26s

 160/1000 [===>..........................] - ETA: 21s

...

 896/1000 [=========================>....] - ETA: 0s

 928/1000 [==========================>...] - ETA: 0s

 960/1000 [===========================>..] - ETA: 0s

 992/1000 [============================>.] - ETA: 0s

1000/1000 [==============================] - 7s 7ms/step


In [25]:
print(sents[:100])
# print(test_sequences[0])
target=[]
for i in range(20):
    words= [tokenizer.index_word[x] for x in test_sequences[50+i]]
    target.append(words)
print(target[:20])

['a woman standing in a bathroom next to a cat', 'a man is laying on his couch while preparing a cell phone', 'a woman in a bathroom preparing food', 'three men in a kitchen with a cat', 'a woman walking in the rain with a cat', 'a woman standing in a bathroom next to a car', 'a woman in a kitchen preparing food', 'a cat in a commercial bathroom on a couch', 'a woman in a kitchen preparing food', 'a woman in a kitchen preparing food', 'a man walking in the kitchen next to a car', 'a woman in a kitchen preparing a meal', 'a woman in a kitchen preparing food', 'a woman standing in a bathroom next to a car', 'a man in a restaurant kitchen with a dog', 'a man in a restaurant with a cat', 'a woman in a kitchen preparing a meal', 'a woman sitting on a couch in a bathroom', 'a man in a kitchen preparing a cat', 'a woman in a kitchen preparing food', 'a woman sitting in a kitchen next to a cat', 'a woman sitting on a couch while talking on a cell phone', 'a man on a laptop in the middle of a k

In [27]:
start=20
print(image_name_train[start])
# img=load_image(image_name_train[0])
img=cv2.imread(image_name_train[start]) 
print(caption_train[start])
# img=cv2.imshow('image',img)  
cv2.imshow('ImageWindow', img)  
cv2.waitKey(0)
cv2.destroyAllWindows()


/home/yiyang/project/caption/caption/data/val2014/COCO_val2014_000000027517.jpg
<start> the broken computer could of warned us about the future cold front <end>


In [13]:
#train_model
def model_train(vocab_size, max_len, units):
    image_input=Input(shape=(299,299,3),dtype=tf.float32)
    seq_input= Input(shape=(max_len,),dtype=tf.int32)
    senti_input= Input(shape=(1,),dtype=tf.int32)
    
    image_feature=image_model(image_input)
    image_feature=Dropout(rate=0.5)(image_feature)
    
    embedded_image= Dense(embedding_size,name="img_embedding",use_bias=False)(image_feature)
    # print(embedded_image.shape,"embedded_img")
    
    embedded_image= BatchNormalization()(embedded_image)
    
    expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))
    embedded_image=expand_dim(embedded_image)
    init_state_c= Input(shape=(units,))
    init_state_h= Input(shape=(units,))

    embedded_seq= Embedding(input_dim=vocab_size,output_dim=units,mask_zero=True,name="seq_input_embedding")(seq_input)
    # print(embedded_seq.shape,"seq_shape")
    embedded_seq=Dropout(rate=0.5)(embedded_seq)
    
    output_embedding=Dense(vocab_size,activation='softmax')
    softmax_layer=Lambda (lambda w: tf.argmax(w,1))
    
    LSTM_layer= keras.layers.LSTM(units, return_sequences=True, 
                             return_state=True,
                             recurrent_initializer='glorot_uniform',
                             dropout=0.5,name="lstm_layer")


    output,state_c, state_h = LSTM_layer(embedded_image,initial_state=[init_state_c,init_state_h])
    
    lstm_output , output, state_h= LSTM_layer(embedded_seq,initial_state=[state_c,state_h])
    # print(lstm_output.shape,"lstm_output")
    logits= output_embedding(lstm_output)
    # print(logits.shape,"shape_logit")
    remove_last= Lambda (lambda w: tf.slice(w,[0,0,0],[-1,max_length-1,-1]))
    logits_2= remove_last(logits)
    # print(logits_2.shape,"logit_shape")
    # logits_3= Reshape(target_shape=(512,))(logits_2)
    # print(logits_3.shape,"stack lstm")
    train_model= keras.Model([image_input,seq_input,senti_input,init_state_c,init_state_h],logits_2)
    train_model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam')

    # print(train_model.output_shape)
    
    return train_model



In [14]:


model_train_1= model_train(vocab_size=vocab_size,max_len=max_length,units=units)

W1001 15:57:59.837060 139620787283776 deprecation.py:506] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W1001 15:58:00.112403 139620787283776 deprecation.py:323] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W1001 15:58:00.141337 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [15]:
#inference model
def model_inference(vocab_size, max_len, units):

    image_input=Input(shape=(299,299,3),dtype=tf.float32)
    seq_input= Input(shape=(1,),dtype=tf.int32)
    senti_input= Input(shape=(1,),dtype=tf.int32)
    
    output_embedding= Dense(vocab_size,activation='softmax')
    
    image_feature=image_model(image_input) 
    embedded_image= Dense(embedding_size,use_bias=False,name="img_embedding")(image_feature)
    embedded_image= BatchNormalization()(embedded_image)
    expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))
    embedded_image=expand_dim(embedded_image)
    
    # print(embedded_image.shape,"embedded_img")
    
    embedded_seq= Embedding(input_dim=vocab_size,output_dim=units,mask_zero=True,name="seq_input_embedding")(seq_input)
    # print(embedded_seq.shape,"seq_shape")
    
    
    argmax_layer= Lambda (lambda w: tf.argmax(w,-1))
    # print(embedded_image.shape,"image_shape")
    
    
    LSTM_unit= keras.layers.LSTM(units, return_sequences=False, 
                                 return_state=True,
                                 recurrent_initializer='glorot_uniform',
                                 dropout=0.5,name="lstm_layer")
    
    init_state_c= Input(shape=(units,))
    init_state_h= Input(shape=(units,))
    
    output,state_c, state_h = LSTM_unit(embedded_image,initial_state=[init_state_c,init_state_h])
    print(output.shape,"output")
    # print(state_c.shape,"state_c")
    # print(state_h.shape,"state_h")
    
    # squeeze= Lambda (lambda w: tf.squeeze(w, 1))
    
    outputs=[]
    for j in range(max_len):
        output, state_c, state_h= LSTM_unit(embedded_seq,initial_state=[output,state_h])
        
        expand_output= expand_dim(output)
        # print(output.shape,"lstm_out")
        # print(expand_output.shape,"expand")
        softmax_output= output_embedding(expand_output)
        # print(softmax_output.shape,"softmax")
        next_input= argmax_layer(softmax_output)
        outputs.append(next_input)
        # print(next_input.shape,"next input shape")
        # next_input= expand_dim(next_input)
        
        embedded_seq= Embedding(input_dim=vocab_size,output_dim=units)(next_input)
        
        # print(next_input.shape,"next input shape")
        
        
        # print(embedded_seq.shape,"loop seq shape")
        # state_c= output
    
    # print(outputs[0].shape,"output shape")
    
    inference_model= keras.Model([image_input,seq_input,senti_input,init_state_c,init_state_h],outputs)
    
    
    # print(inference_model.output_shape)
    return inference_model

In [16]:
model_inf_1= model_inference(vocab_size,max_length,units)


(?, 512) output


In [17]:
#inference model - image embedding lstm
def image_dense_lstm(embedding_size):
    
    image_input=Input(shape=(299,299,3),dtype=tf.float32)
    image_feature=image_model(image_input) 
    embedded_image= Dense(embedding_size,use_bias=False,name="img_embedding")(image_feature)
    embedded_image= BatchNormalization()(embedded_image)
    expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))
    embedded_image=expand_dim(embedded_image)

    init_state_c= Input(shape=(units,))
    init_state_h= Input(shape=(units,))
    LSTM_unit= keras.layers.LSTM(units, return_sequences=False, 
                                 return_state=True,
                                 recurrent_initializer='glorot_uniform',
                                 dropout=0.5,name="lstm_layer")
    
    output,state_c, state_h = LSTM_unit(embedded_image,initial_state=[init_state_c,init_state_h])
    # print(output.shape,"output")

    return keras.Model(inputs=[image_input, init_state_c, init_state_h], outputs=[state_c, state_h])


In [18]:
#inference_model - text_embedding lstm
def text_emb_lstm(vocab_size,units):
    seq_input= Input(shape=(1,),dtype=tf.int32)
    embedded_seq= Embedding(input_dim=vocab_size,output_dim=units,mask_zero=True,name="seq_input_embedding")(seq_input)
    # print(embedded_seq.shape,"seq_shape")
    LSTM_unit= keras.layers.LSTM(units, return_sequences=False, 
                                 return_state=True,
                                 recurrent_initializer='glorot_uniform',
                                 dropout=0.5,name="lstm_layer")
    init_state_c= Input(shape=(units,))
    init_state_h= Input(shape=(units,))
    
    output_embedding= Dense(vocab_size,activation='softmax')
    expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))
    squeeze_dim= Lambda (lambda w: tf.squeeze(w,1))
    output, state_c, state_h= LSTM_unit(embedded_seq,initial_state=[init_state_c,init_state_h])
    output= expand_dim(output)
    lstm_output= output_embedding(output)
    lstm_output= squeeze_dim(lstm_output)
    
    return keras.Model(inputs=[seq_input,init_state_c,init_state_h],outputs=[lstm_output,state_c,state_h])

    
    

In [19]:
input_imgs=[]
start=0
for j in tqdm(range(8000)):
    img= load_image( image_name_train[start+j])
    input_imgs.append(img)
input_imgs= np.array(input_imgs)
input_sentis= np.array(senti_train[:8000])
input_sequences= np.array(cap_vector_train[:8000])

print(input_sequences.shape,"input_seq")
target= input_sequences[:,1:]

target= np.expand_dims(target,axis=-1)
print(target.shape,"target")
state_c= np.zeros(shape=(8000,512))
state_h= np.zeros(shape=(8000,512))

  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 25/8000 [00:00<00:32, 245.77it/s]

  1%|          | 51/8000 [00:00<00:31, 248.45it/s]

  1%|          | 71/8000 [00:00<00:34, 229.02it/s]

  1%|          | 97/8000 [00:00<00:33, 235.60it/s]

...

 99%|█████████▉| 7927/8000 [00:31<00:00, 237.49it/s]

 99%|█████████▉| 7951/8000 [00:32<00:00, 236.49it/s]

100%|█████████▉| 7977/8000 [00:32<00:00, 242.96it/s]

100%|██████████| 8000/8000 [00:32<00:00, 247.77it/s]

(8000, 20) input_seq
(8000, 19, 1) target


In [23]:
# model_train_1.fit(x=[input_imgs,input_sequences,input_sentis,state_c,state_h],y=target, batch_size=BATCH_SIZE,verbose=1,epochs=10)
# 
# model_train_1.save_weights("models/model6.h5")


Epoch 1/10


  32/8000 [..............................] - ETA: 34:08 - loss: 8.4111

  64/8000 [..............................] - ETA: 17:11 - loss: 8.4064

  96/8000 [..............................] - ETA: 11:32 - loss: 8.3992

 128/8000 [..............................] - ETA: 8:42 - loss: 8.3902 

...

7872/8000 [============================>.] - ETA: 0s - loss: 1.7503

7904/8000 [============================>.] - ETA: 0s - loss: 1.7501

7936/8000 [============================>.] - ETA: 0s - loss: 1.7500

7968/8000 [============================>.] - ETA: 0s - loss: 1.7501

8000/8000 [==============================] - 23s 3ms/step - loss: 1.7503


In [20]:
model_train_1.load_weights("models/model6.h5")
model_train_1.fit(x=[input_imgs,input_sequences,input_sentis,state_c,state_h],y=target, batch_size=BATCH_SIZE,verbose=2,epochs=20)

model_train_1.save_weights("models/model6.h5")


Epoch 1/20


 - 28s - loss: 0.2876


Epoch 2/20


 - 22s - loss: 0.2811


Epoch 3/20


...

 - 22s - loss: 0.2691


Epoch 19/20


 - 22s - loss: 0.2684


Epoch 20/20


 - 22s - loss: 0.2671


In [21]:
test_imgs=[]
start=0
for j in tqdm(range(8000,8200)):
    img= load_image( image_name_train[start+j])
    test_imgs.append(img)
test_imgs= np.array(test_imgs)
test_sentis= np.array(senti_train[8000:8200])
test_sequences= np.array(cap_vector_train[8000:8200])
test_seq= np.ones(shape=(200,1))
test_seq= 3* test_seq
test_out=[]


  0%|          | 0/200 [00:00<?, ?it/s]

 12%|█▏        | 23/200 [00:00<00:00, 225.10it/s]

 23%|██▎       | 46/200 [00:00<00:00, 226.20it/s]

 35%|███▌      | 70/200 [00:00<00:00, 228.17it/s]

 44%|████▍     | 89/200 [00:00<00:00, 213.69it/s]

 56%|█████▌    | 112/200 [00:00<00:00, 215.96it/s]

 68%|██████▊   | 137/200 [00:00<00:00, 222.48it/s]

 80%|████████  | 161/200 [00:00<00:00, 224.36it/s]

 92%|█████████▎| 185/200 [00:00<00:00, 226.57it/s]

100%|██████████| 200/200 [00:00<00:00, 227.62it/s]

In [22]:
model_inf_1.load_weights("models/model7.h5",by_name=True,skip_mismatch=True)
state_c= np.zeros(shape=(200,512))
state_h= np.zeros(shape=(200,512))
preds= model_inf_1.predict(x=[test_imgs[:200],test_seq[:200],test_sentis[:200],state_c,state_h],batch_size=32,verbose=1)


 32/200 [===>..........................] - ETA: 14s

 64/200 [========>.....................] - ETA: 6s 

 96/200 [=============>................] - ETA: 3s

128/200 [==================>...........] - ETA: 1s

160/200 [=======================>......] - ETA: 0s

192/200 [===========================>..] - ETA: 0s

200/200 [==============================] - 4s 18ms/step


In [23]:
print(preds[0].shape)
preds1= np.array(preds)
print(preds1.shape)
preds2= np.squeeze(preds1,axis=2)
preds_t= np.transpose(preds2)
print(preds_t.shape)
# print(test_seq[:10])
# print(preds_t)
len1=200

sent=[]
for i in range(len1):
    words= [tokenizer.index_word[x] for x in preds_t[i]]
    sent.append(words)
print(sent[:20])



(200, 1)
(20, 200, 1)
(200, 20)
[['blocking', 'yard', 'yard', 'couch', 'couch', 'canned', 'canned', 'canned', 'canned', 'walk', 'walk', 'walk', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports'], ['blocking', 'yard', 'yard', 'couch', 'couch', 'canned', 'canned', 'canned', 'canned', 'walk', 'walk', 'walk', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports', 'sports'], ['her', 'spend', 'inviting', 'inviting', 'load', 'consisted', 'spend', 'load', 'warehouse', 'spend', 'load', 'spend', 'load', 'load', 'load', 'warehouse', 'guarding', 'guarding', 'guarding', 'guarding'], ['her', 'spend', 'inviting', 'inviting', 'load', 'consisted', 'spend', 'load', 'warehouse', 'spend', 'load', 'spend', 'load', 'load', 'load', 'warehouse', 'guarding', 'guarding', 'guarding', 'guarding'], ['her', 'spend', 'inviting', 'inviting', 'load', 'consisted', 'spend', 'load', 'warehouse', 'spend', 'load', 'spend', 'load', 'load', 'load', 'warehouse', 'guarding', 'guardin

In [27]:
print(test_sequences[:20])
sent=[]
for i in range(len1):
    words= [tokenizer.index_word[x] for x in test_sequences[i]]
    sent.append(words)
print(sent[:20])

[[   3    2  270   22    8    5  882    7    2  154   55  241    2   17
   565    4    0    0    0    0]
 [   3    2  270   22    8    5  882    9   12   95  565    4    0    0
     0    0    0    0    0    0]
 [   3   12  198   42   77    7    5  180   29  507    2 1972    4    0
     0    0    0    0    0    0]
 [   3    2  425   42 1992   90   48  180   29    7   37    6    2   36
     4    0    0    0    0    0]
 [   3    2   16   42   10   26    7 2051  296    2  191    4    0    0
     0    0    0    0    0    0]
 [   3   21  395  487  426   23    7  210    9  105   15    8    5  316
     4    0    0    0    0    0]
 [   3    2  106   30   10   65    7    5   17   19    4    0    0    0
     0    0    0    0    0    0]
 [   3    2  106   10  322    7    2  420   54    4    0    0    0    0
     0    0    0    0    0    0]
 [   3    2  172   49    9    2  564 1254    8   28   65   33   49  257
     4    0    0    0    0    0]
 [   3    5  100   78    2   17  203    7    5  103 217

In [24]:
def beam_search( decoder_model,units, init_state_c , init_state_h, tokenizer, beam_width, max_len, alpha = 0.7):
    

    assert(init_state_c.shape == (1, units) and init_state_h.shape == (1, units) and isinstance(beam_width, int) and
             beam_width > 0 and max_len > 0)

    # === first step ====
    start_word = np.array([tokenizer.word_index['<start>']])

    output, a, c = decoder_model.predict([start_word,init_state_c , init_state_h], verbose=0)
    # output=np.squeeze(output,axis=1)
    print(output.shape,"lstm output shape")
    assert(len(output.shape)==2 and beam_width<=output.shape[1])

    # === define data structure and initialization====
    
    seeds = np.argpartition(-output, beam_width, axis=-1)[0, :beam_width]
    start_words = np.array(seeds)
    next_activates = np.repeat(a, beam_width, axis = 0)
    next_cells = np.repeat(c, beam_width, axis = 0)

    scores = [math.log(output[0, i]) for i in seeds]
    routes = [[i] for i in seeds]
    res = {'scores':[], 'routes':[]}

    # === search ====
    for i in range(max_len-1):

        outputs, activations, cells = decoder_model.predict([start_words, next_activates, next_cells], 
                                                            verbose=0)
        # outputs=np.squeeze(outputs,axis=1)
        # pick <beam_width> highest scores from every route as a candidate
        candidates = np.argpartition(-outputs, beam_width, axis=-1)[:,:beam_width]
        # r <----> i-th in scores and routes, c is the index of vocabulary
        candidates = [(r, c) for r in range(candidates.shape[0]) for c in candidates[r,:]]
        # calculate score according to the candidates
        candidates_scores = np.array([scores[r] + math.log(outputs[r, c]) for r, c in candidates])
        # consider the length of the current sentence
        #weigthed_scores = 1/(i+1)**alpha * candidates_scores
        if beam_width < len(candidates):
            choosen_candidates = np.argpartition(-candidates_scores, beam_width)[:beam_width]
        else:
            choosen_candidates = np.arange(0, len(candidates))

        # update scores, routes
        # construct new start_words, activations, cells
        start_words = []
        next_activates = []
        next_cells = []
        updated_scores = []
        updated_routes = []
        for idx in choosen_candidates:
            r, c = candidates[idx]
            if c == tokenizer.word_index['<end>']:
                res['routes'].append(routes[r])
                
                if i != 0:
                    res['scores'].append(1/len(routes[r])**alpha * candidates_scores[idx])
                else:
                    res['scores'].append(-math.inf)
                
                beam_width -= 1
            else:
                start_words.append(c)
                next_activates.append(activations[r, :])
                next_cells.append(cells[r, :])

                updated_scores.append(candidates_scores[idx])
                updated_routes.append(routes[r]+[c])

        start_words = np.array(start_words)
        next_activates = np.array(next_activates)
        next_cells = np.array(next_cells)
        scores = updated_scores
        routes = updated_routes

        if beam_width <= 0:
            break

    res['scores'] += [1/len(routes[i])**alpha * scores[i] for i in range(len(scores))]
    res['routes'] += routes

    return res


In [25]:
model_inf_img=image_dense_lstm(embedding_size)
model_inf_seq= text_emb_lstm(vocab_size,units)
model_inf_img.load_weights('models/model6.h5',by_name=True,skip_mismatch=True)
model_inf_seq.load_weights('models/model6.h5',by_name=True,skip_mismatch=True)

In [39]:
img0= test_imgs[10]
init_state_c= np.zeros((1,512))
init_state_h= np.zeros((1,512))
state_c, state_h = model_inf_img.predict(x=[[img0],init_state_c,init_state_h])
print(state_c.shape)

(1, 4500) lstm output shape


In [40]:

res = beam_search( model_inf_seq,units,init_state_c,init_state_h,tokenizer,beam_width=5,max_len=max_length,alpha=0.7)
best_index= np.argmax(res)
caption=tokenizer.sequences_to_texts([res])


(1, 4500) lstm output shape


In [41]:
print(caption)

['<unk> <unk>']


In [34]:
class CNN_Encoder(keras.Model):
    
    def __init__(self, embedding_size):
        super(CNN_Encoder, self).__init__()
        
        # self.pre_processing= image_model
        self.img_embedding = Dense(embedding_size)
        self.expand_dim= Lambda (lambda w: tf.expand_dims(w, 1))

    def call(self, x):
        #input shape : (299,299,3) out_put shape: (n,1,512)
        x = image_model(x)
        x = self.img_embedding(x)
        x= self.expand_dim(x)
        return x
    

In [ ]:
class RNN_Decoder(keras.Model):
  def __init__(self, embedding_size, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding= Embedding(vocab_size, embedding_size)
    self.lstm = keras.layers.LSTM(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    
    self.output_embedding= Dense(vocab_size,activation='softmax')
    

  def call(self, x, state_c, state_h):
    

    # x shape after passing through embedding == (batch_size, 1, embedding_size)
    x = self.embedding(x)


    # passing the concatenated vector to the GRU
    output, state_c, state_h = self.lstm(x, initial_state=[state_c, state_h])

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.output_embedding(x)

    return x, state_c, state_h

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))
  

In [ ]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [ ]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [ ]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
